In [4]:
from subprocess import Popen, PIPE
import platform

In [20]:
def ping_many_windows(request, errors=False):
    """ Ping many ips """

    # every `interval` seconds, launch `count` pings, abort if returns take longer than `timeout` seconds
    interval = .25 # can be float
    timeout = 3 # must be int
    count = 3

    commands = [f"ping /w {timeout*1000} /n {count} {ip}" for ip in request.values()]

    # Check if running, return if so

    request['procs'] = []
    request['returns'] = ['-- ms' for ip in commands]

    for cmd in commands:

        if errors:
            stderr = PIPE
        else:
            stderr = None

        request['procs'] += [Popen(cmd, stdout=PIPE, stderr=stderr, shell=True)]

    return request


In [34]:
req = ping_many_windows({0:"7.0.0.1"})
req

{0: '7.0.0.1',
 'procs': [<subprocess.Popen at 0x25b07f54648>],
 'returns': ['-- ms']}

In [35]:
def parse_ping_windows(ping):
    """ macOS ping parsing """

    value = "-- ms"

    print('...',ping)
    
    if ping is None or '100.0% packet loss' in ping:
        return value

    blob = ping.split('\n')[-2] # "line after ping statistics"

    if blob == '':
        return value

    # get avg from roundtrip min/avg/max/stddev line
    value = (blob.split(' = ')[-1]).strip() # includes ms

    return value

def get_pings_windows(request):
    """ Get the returns of pinging many ips """

    for i, p in enumerate(request['procs']):

        if p.poll() is not None:

            # Done running, get output
            try:
                out,err = p.communicate()
                request['returns'][i] = parse_ping_windows(str(out,'utf-8'))
                p.kill()

            except ValueError:
                # p has already been killed
                pass
        else:
            print("...p.poll was none")

    return request

In [37]:
get_pings_windows(req)

... 
Pinging 7.0.0.1 with 32 bytes of data:
Reply from 7.0.0.1: bytes=32 time=2ms TTL=64
Reply from 7.0.0.1: bytes=32 time=3ms TTL=64
Reply from 7.0.0.1: bytes=32 time=2ms TTL=64

Ping statistics for 7.0.0.1:
    Packets: Sent = 3, Received = 3, Lost = 0 (0% loss),
Approximate round trip times in milli-seconds:
    Minimum = 2ms, Maximum = 3ms, Average = 2ms



{0: '7.0.0.1',
 'procs': [<subprocess.Popen at 0x25b07f54648>],
 'returns': ['2ms ms']}

In [38]:
platform.system().lower()

'windows'